In [1]:
import xml.etree.ElementTree as ET
import os
import yaml
import cv2
import random
import numpy as np

train_split = 'UCF101_Action_detection_splits/trainlist01.txt'
test_split = 'UCF101_Action_detection_splits/testlist01.txt'
annotation_path = 'annotation/'
xml_annotation_path = 'xml_annotation/'
image_path = 'images/'


        

        

filelist=[]

for root,subdir,dirs in os.walk(annotation_path):
    for file_name in dirs:
#        if file_name == 'annotation/TennisSwing/v_TennisSwing_g09_c01.xgtf' or \
#        file_name == 'annotation/TennisSwing/v_TennisSwing_g09_c02.xgtf':
#            continue
        filelist.append(os.path.join(root,file_name))
        #print os.path.join(root,file_name)

class annotation: 
    def __init__(self,count,frame_num):
        self.img_num = count
        self.label_id = []
        self.x = []
        self.y = []
        self.width = []
        self.height = []
        self.flag = []
        self.frame = []
        self.start = 1
        self.frame_num = frame_num
        self.ratio = float(count)/float(frame_num)
        self.img_list = []
    
    def set_img_list(self,path):
        for root,subdir,dirs in os.walk(path):
            for file_name in dirs:
                img_path = os.path.join(root,file_name)
                #print os.path.join(root,file_name)
                self.img_list.append(img_path)


    
        
    def add_Anno(self,x,y,framespan,width,height,label_id):
        self.frame_num = int(int(framespan.split(':')[1])*self.ratio)
        id1 = int(int(framespan.split(':')[0])*self.ratio)
        #print 'id1 = ', id1
        #print 'id2 = ', self.frame_num
        for i in range(id1,self.frame_num+1):
            self.flag.append(0)
            self.x.append(int(x))
            #print 'x = ',x
            self.y.append(int(y))
            #print 'y = ',y
            self.width.append(int(width))
            #print 'width = ',width
            self.height.append(int(height))
            #print 'height = ',height
            self.label_id.append(int(label_id))
            self.frame.append(int(i))

    def add_frame_span(self,framespan,ids):
            #print 'call add frame span'
            #print framespan,ids
            
            frame_start = int(int(framespan.split(':')[0])*self.ratio)
            frame_end = int(int(framespan.split(':')[1])*self.ratio)
            
            #print 'frame_start ',frame_start
            #print 'frame_end ',frame_end
            for i in range(0,len(self.x)):
                #print frame_start, 'i = ', i , frame_end
                if frame_start + self.frame_num*ids <= i <= frame_end + self.frame_num*ids :
                    tmp_flag = 1
                else:
                    tmp_flag = 0
            
                if tmp_flag == 1 and self.label_id[i]==ids:
                    #print 'change flag',self.label_id[i]
                    self.flag[i] = tmp_flag
                    
    def get_Label(self):
        tmp = []
        
        for i in range(0,len(self.label_id)):
              tmp.append([self.label_id[i],self.frame[i],self.x[i],self.y[i],self.width[i],self.height[i],self.flag[i]])
                      
        return tmp


In [ ]:
class list_pair:
 
    def __init__(self,split_path):
        self.split_path = split_path
        self.pairs = {}
        self.img_list = []
        self.Parse_split()

    def Parse_split(self):
        with open(self.split_path) as f:
            global image_path
            content = f.read().splitlines()
            for lines in content:
                pair = lines.split(' ')
                full_path = image_path + pair[0].split('.')[0]
                self.img_list.append(full_path)
                self.pairs[full_path] = pair[1]

                

        

img_list = list_pair(train_split)        

print train_split

ann_img_pair = {}

for img_name in img_list.img_list:
    print img_name
    
    #print img_list.pairs[img_name]

In [2]:
 
def parse(path,count): 
    folder_name = path.split('/')[1].split('.')[0]
    tree = ET.parse(path)
    root = tree.getroot()
    tree_string = ET.tostring(root, encoding='utf8', method='xml')
    root = ET.fromstring(tree_string)

    length = 0

    obj_num = 0
    save_flag = 0
    Tframe_num = []
    for child in root.iter():
        if child.tag == '{http://lamp.cfar.umd.edu/viper#}object':
            obj_num = max(obj_num,int(child.attrib['id']))
        if child.tag =='{http://lamp.cfar.umd.edu/viperdata#}dvalue' :
            Tframe_num.append(int(child.attrib['value']))
    
    obj_num +=1    
    #print obj_num
    #print max(Tframe_num)
    obj_idx = range(0,obj_num)
    
    
    annotations = annotation(count,max(Tframe_num))

    flag = 0    
    save_flag = 0
    for child in root.getchildren()[1].iter():
        #print child.tag,child.attrib
        if child.tag =='{http://lamp.cfar.umd.edu/viperdata#}dvalue' :
            flag +=1
        if child.tag =='{http://lamp.cfar.umd.edu/viper#}object' :
            idx = int(child.attrib['id'])
            #print idx
        if child.tag =='{http://lamp.cfar.umd.edu/viper#}sourcefile' :
            #print child.attrib['filename'].split('.')[1]
            if child.attrib['filename'].split('.')[1]== 'mpg' or child.attrib['filename'].split('.')[1]== 'jpeg':
                #print 'start save'
                save_flag = 1
        #{http://lamp.cfar.umd.edu/viperdata#}bbox
        #if (flag%2==1) and (child.tag == '{http://lamp.cfar.umd.edu/viperdata#}bbox'):
        if (child.tag == '{http://lamp.cfar.umd.edu/viperdata#}bbox') and save_flag == 1:
            #print child.attrib['framespan'],child.attrib['x'],child.attrib['y'],child.attrib['height'],child.attrib['width']        
            #print child.tag, child.attrib
            annotations.add_Anno(child.attrib['x'],child.attrib['y'],child.attrib['framespan'],child.attrib['width'],child.attrib['height'],idx)
    if save_flag == 0:
        print path
        print "=================================================================================="
    flag2 = 0
    save_flag = 0
    for child in root.getchildren()[1].iter():
        if child.tag =='{http://lamp.cfar.umd.edu/viper#}object' :
            ids = int(child.attrib['id'])
        if child.tag =='{http://lamp.cfar.umd.edu/viperdata#}dvalue' :
            flag2 +=1
        if child.tag =='{http://lamp.cfar.umd.edu/viper#}sourcefile' :
            #print child.attrib['filename'].split('.')[1]
            if child.attrib['filename'].split('.')[1]== 'mpg' or child.attrib['filename'].split('.')[1]== 'jpeg':
                save_flag = 1                
        if child.tag =='{http://lamp.cfar.umd.edu/viperdata#}bvalue' and save_flag == 1:
            #if (child.attrib['value']) == 'true' and (flag2%2 ==1):
            if child.attrib['value']=='true':
                happen_span = child.attrib['framespan']
                annotations.add_frame_span(happen_span,ids)
                
    return annotations  


In [3]:

annotation_list = {}
xml_annotation_folder = {}
label_list = []

for file_name in filelist:
    folder_name = file_name.split('/')[2].split('.')[0]
    video_name = folder_name
    label_list.append(video_name)
    print "generating " + video_name + ""
    xml_annotation_folder_name = xml_annotation_path+file_name.split('/')[1]+'/'+folder_name
    folder_name = 'images/'+file_name.split('/')[1]+'/'+folder_name
    xml_annotation_folder[video_name] = xml_annotation_folder_name
    print folder_name
    count = 0
    for root,subdir,files in os.walk(folder_name):
        fileLength = len(files)
        if fileLength != 0:
            count = count + fileLength
    annotation_list[video_name]=parse(file_name,count)
    #print max(annotation_list[video_name].label_id)
  


generating v_Basketball_g01_c01
images/Basketball/v_Basketball_g01_c01
generating v_Basketball_g01_c02
images/Basketball/v_Basketball_g01_c02
generating v_Basketball_g01_c03
images/Basketball/v_Basketball_g01_c03
generating v_Basketball_g01_c04
images/Basketball/v_Basketball_g01_c04


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:30: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:53: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


generating v_Basketball_g01_c05
images/Basketball/v_Basketball_g01_c05
generating v_Basketball_g01_c06
images/Basketball/v_Basketball_g01_c06
generating v_Basketball_g01_c07
images/Basketball/v_Basketball_g01_c07
generating v_Basketball_g02_c01
images/Basketball/v_Basketball_g02_c01
generating v_Basketball_g02_c02
images/Basketball/v_Basketball_g02_c02
generating v_Basketball_g02_c03
images/Basketball/v_Basketball_g02_c03
generating v_Basketball_g02_c04
images/Basketball/v_Basketball_g02_c04
generating v_Basketball_g02_c05
images/Basketball/v_Basketball_g02_c05
generating v_Basketball_g02_c06
images/Basketball/v_Basketball_g02_c06
generating v_Basketball_g03_c01
images/Basketball/v_Basketball_g03_c01
generating v_Basketball_g03_c02
images/Basketball/v_Basketball_g03_c02
generating v_Basketball_g03_c03
images/Basketball/v_Basketball_g03_c03
generating v_Basketball_g03_c04
images/Basketball/v_Basketball_g03_c04
generating v_Basketball_g03_c05
images/Basketball/v_Basketball_g03_c05
genera

In [ ]:
debug_name = 'v_Diving_g07_c01'

debug_filename = 'annotation/Basketball/v_Basketball_g01_c01.xgtf'

frame_dict = {}

debug_anno = parse(debug_filename,119)

ids = label_ids = int(max(debug_anno.label_id))
print ids
single_anno = debug_anno.get_Label()

#print debug_anno.get_Label()

frame_num = int(max(np.array(debug_anno.get_Label())[:,1]))
print frame_num

anno = np.array(debug_anno.get_Label())
#print anno[anno[:,1]==2]

ab = debug_anno.get_Label()
for a in ab:
    print a
#for idx in xrange(1,frame_num+1):
 #   print anno
    #print "--"
    




In [ ]:
for file_name in label_list:
    print "images/"+file_name.split('_')[1]+"/"+file_name

In [4]:


for file_name in label_list:
    #print file_name
    rands = random.randint(1,50)
    print "generating " + file_name + " xml format"
    if rands > 0:
        xml_path = xml_annotation_folder[file_name]
        anno_file = annotation_list[file_name]
        label_ids = max(anno_file.label_id)
        frame_num = int(max(np.array(anno_file.get_Label())[:,1]))
        anno = np.array(anno_file.get_Label())
        for idx in xrange(1,frame_num+1):
                if not os.path.exists(os.path.dirname(os.path.join(xml_path, str(idx) + '.xml'))):
                    os.makedirs(os.path.dirname(os.path.join(xml_path,str(idx) + '.xml')))
                with open(os.path.join(xml_path, str(idx) + '.xml'), 'w') as f:
                    print >>f, "<annotation>"
                    print >>f, "<folder>", "images/"+file_name.split('_')[1]+"/"+file_name, "</folder>"
                    print >>f, "<filename>", str(idx)+".jpg", "</filename>"
                    print >>f, "<size>"
                    print >>f, "<width>", "320", "</width>"
                    print >>f, "<height>", "240", "</height>"
                    print >>f, "<depth>", "3", "</depth>"
                    print >>f, "</size>"
                    for bbox in anno[anno[:,1]==idx]:
                        print >>f, "<object>"
                        print >>f, "<name>",file_name.split('_')[1],"</name>"
                        print >>f, "<pose>Unspecified</pose>"
                        print >>f, "<truncated>0</truncated>"
                        print >>f, "<difficult>0</difficult>"
                        print >>f, "<action>",bbox[6],"</action>" 
                        print >>f, "<bndbox>"
                        print >>f, "<xmin>", str(bbox[2]), "</xmin>"
                        print >>f, "<xmax>", str(int(bbox[2])+int(bbox[4])), "</xmax>"
                        print >>f, "<ymin>", str(bbox[3]), "</ymin>"
                        print >>f, "<ymax>", str(int(bbox[3])+int(bbox[5])), "</ymax>"
                        print >>f, "</bndbox>"
                        print >>f, "</object>"
                        print >>f, "</annotation>"                 
      


generating v_Basketball_g01_c01 xml format
generating v_Basketball_g01_c02 xml format
generating v_Basketball_g01_c03 xml format
generating v_Basketball_g01_c04 xml format
generating v_Basketball_g01_c05 xml format
generating v_Basketball_g01_c06 xml format
generating v_Basketball_g01_c07 xml format
generating v_Basketball_g02_c01 xml format
generating v_Basketball_g02_c02 xml format
generating v_Basketball_g02_c03 xml format
generating v_Basketball_g02_c04 xml format
generating v_Basketball_g02_c05 xml format
generating v_Basketball_g02_c06 xml format
generating v_Basketball_g03_c01 xml format
generating v_Basketball_g03_c02 xml format
generating v_Basketball_g03_c03 xml format
generating v_Basketball_g03_c04 xml format
generating v_Basketball_g03_c05 xml format
generating v_Basketball_g03_c06 xml format
generating v_Basketball_g04_c01 xml format
generating v_Basketball_g04_c02 xml format
generating v_Basketball_g04_c03 xml format
generating v_Basketball_g04_c04 xml format
generating 

In [ ]:
for file_name in label_list:
    #print file_name
    rands = random.randint(1,50)
    print "generating " + file_name + " xml format"
    if rands > 0:
        xml_path = xml_annotation_folder[file_name]
        anno_file = annotation_list[file_name]
        label_ids = max(anno_file.label_id)
        frame_num = max(anno_file.get_Label()[1])
        anno = np.array(anno_file.get_Label())
        for idx in xrange(1,frame_num+1):
                #with open(os.path.join(xml_path, str(frame) + '.xml'), 'w') as f:
            print  "<annotation>"
            print  "<folder>", "images/"+file_name.split('_')[1]+"/"+file_name, "</folder>"
            print  "<filename>", str(idx)+".jpg", "</filename>"
            print  "<size>"
            print  "<width>", "320", "</width>"
            print  "<height>", "240", "</height>"
            print  "<depth>", "3", "</depth>"
            print  "</size>"
            for bbox in anno[anno[:,1]==idx]:
                print  "<object>"
                print  "<name>", file_name.split('_')[1], "</name>"
                print  "<pose>Unspecified</pose>"
                print  "<truncated>0</truncated>"
                print  "<difficult>0</difficult>"
                print  "<action>",bbox[6],"</action>"                
                print  "<bndbox>"
                print  "<xmin>", str(bbox[2]), "</xmin>"
                print  "<xmax>", str(int(bbox[2])+int(bbox[4])), "</xmax>"
                print  "<ymin>", str(bbox[3]), "</ymin>"
                print  "<ymax>", str(int(bbox[3])+int(bbox[5])), "</ymax>"
                print  "</bndbox>"
                print  "</object>"
                print  "</annotation>"        
            break
        break